In [1]:
import telebot
import time # Librería para hacer que el programa que controla el bot no se acabe.
TOKEN = '809030422:AAEGpaQFqNdgmJLfJZbuuQvouLhjaG7GFYc' # Nuestro token del bot.
AYUDA = 'Puedes utilizar los siguientes comandos : \n\n/ayuda - Guia para utilizar el bot. \n/info - Informacion De interes \n/hola - Saludo del Bot \n/piensa3D - Informacion sobre Piensa3D \n\n'

In [2]:
bot = telebot.TeleBot(TOKEN)

In [3]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler

In [4]:
#Creamos nuestra instancia "mi_bot" a partir de ese TOKEN
mi_bot = telegram.Bot(token=TOKEN)
mi_bot_updater = Updater(mi_bot.token)

In [5]:
def start(bot=mi_bot, update=mi_bot_updater):
    print("Iniciada conversación: ")
    print(update.message.chat_id)
    bot.sendMessage(chat_id=update.message.chat_id, text="Hola, soy tu bot!!!")

# Handle para la ayuda /?
def ayuda(bot=mi_bot, update=mi_bot_updater):
    print("Solicita ayuda")
    bot.sendMessage(chat_id=update.message.chat_id, text=Ayuda)

In [6]:
#Definimos para cada comando la función que atenderá la peticion
start_handler = CommandHandler('start', start)
ayuda_handler = CommandHandler('?', ayuda)

In [7]:
Clientes = {}
def NuevoCliente(bot=mi_bot, update=mi_bot_updater):
    Nuevo_Cliente = Cliente(bot,update)
    Clientes[update.message.chat_id] = Nuevo_Cliente
    bot.sendMessage(chat_id=update.message.chat_id, text="No tengo mucho que ofrecerte por ahora......")
    Cliente.PrintSeleccion()

In [8]:
class Cliente:
    def __init__(self, bot, update, nombre):
        self.platos_elegidos = {}
        self.precio = 0
        self.ver_alergenos = False
        self.ver_ingredientes = False
        self.nombre = nombre
        self.bot = bot
        self.update = update
        
    def PrintSeleccion(self):
        if len(self.platos_elegidos)==0:
            print('Platos elegidos:')
            mensaje = 'No ha elegido platos. \n'
        else:
            mensaje = 'Platos elegidos:\n'
            for plato in self.platos_elegidos.keys():
                mensaje += ' -'+plato.capitalize()+' x'+str(self.platos_elegidos[plato])+'\n'
                
                if self.ver_ingredientes:
                    Ingredientes = carta.loc[plato,'ingredientes']
                    if len(Ingredientes)> 0:
                        mensaje += 'Ingredientes: '
                        for ing in Ingredientes:
                            mensaje += ing.capitalize() +'. '
                        mensaje += '\n'
                    
                if self.ver_alergenos:
                    Alergenos = carta.loc[plato,'alergenos']
                    if len(Alergenos) > 0:
                        mensaje += 'Alergenos: '
                        for al in Alergenos:
                            mensaje += al.capitalize() +'. '
                        mensaje += '\n'
                    
                
                    
            mensaje += '-'*20+'\n'
            mensaje += 'Precio: '+str(self.precio)+' €'
        # Enviamos la respuesta
        self.bot.sendMessage(chat_id=self.update.message.chat_id, text=mensaje)
        
    def AñadirPlato(self,plato,cantidad):
        # plato --> Plato elegido
        # cantidad --> Veces que se ha pedido
        if plato in self.platos_elegidos.keys():
            self.platos_elegidos[plato] += cantidad
        else:
            self.platos_elegidos[plato] = cantidad #Solo lo ha pedido una vez
        #Actualizamos el precio del pedido
        self.precio += cantidad*carta.loc[plato,'precio']
        
        if cantidad ==1:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, text='Se ha añadido a la cesta una unidad de '+plato)
        else:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, text='Se han añadido a la cesta '+str(cantidad)+' unidades de '+plato)
        
    

In [9]:
def RegistrarCliente(bot=mi_bot, update=mi_bot_updater):
    ClienteId = update.message.chat_id
    if update.message.chat_id in Clientes.keys():
        bot.sendMessage(chat_id=update.message.chat_id, text="Ya se encuentra registrado.")
    else:
        #Introducimos el nombre
        Nuevo_Cliente = Cliente(bot,update,nombre = update.message.from_user.full_name)
        Clientes[update.message.chat_id] = Nuevo_Cliente
        bot.sendMessage(chat_id=update.message.chat_id, text="Se le ha registrado con exito.")

In [10]:
def ayuda(bot=mi_bot, update=mi_bot_updater):
    print('Help!!!')
    bot.sendMessage(chat_id=update.message.chat_id, text=AYUDA)

In [13]:
import pandas as pd

carta = pd.DataFrame({'nombre' : ['Chuletas con patatas','queso frito','Paella Valenciana','Flan','Cocacola'],
                     'precio' : [13,4,5,2,1.2],
                     'categorias' : ['segundo','entrante','primero','postre','bebida'],
                     'ingredientes' : [['Chuletas','patatas'],
                                       ['Queso'],
                                       ['Arroz','cosas'],
                                       ['huevo','cosas que llevan los flanes'],
                                       []],
                     'alergenos' : [[],
                                    ['lactosa'],
                                    ['gluten'],
                                    [],
                                    []]})
carta = carta.set_index('nombre')
carta = carta.sort_index()

In [14]:
carta.loc[carta['categorias'] == 'postre',:]

,precio,categorias,ingredientes,alergenos
nombre,,,,
Flan,2.0,postre,"[huevo, cosas que llevan los flanes]",[]


Definimos la función que imprime los elementos de un df según el formato de la carta:

In [15]:
def MostrarCarta(bot=mi_bot, update=mi_bot_updater):
    respuesta = ''
    for categoria in ['tapa','entrante', 'primero', 'segundo', 'postre', 'bebida']:
        # Definimos el mensaje de la carta
        mensaje = update.message.text
        mensaje = mensaje.lower() #Ponemos todo en minúsculas
        if 'carta' in mensaje or categoria in mensaje: #Apaño para que salga todo cuando se encuentren la palabra 'carta' 
                                                       #y la categoría en el mismo mensage
            respuesta += categoria.capitalize()+':\n'
            df_aux = carta.loc[carta['categorias'] == categoria,:]
            
            if df_aux.shape[0] > 0:
                for fila_i in range(df_aux.shape[0]):
                    respuesta += ' +'+df_aux.index[fila_i].capitalize()+': '+ str(df_aux.iloc[fila_i]['precio'])+' €.\n'
                    
                    if Clientes[update.message.chat_id].ver_ingredientes:
                        ingredientes = df_aux.iloc[fila_i]['ingredientes']
                        if len(ingredientes) > 0:
                            respuesta += 'Ingredientes: '
                            for ing in ingredientes:
                                respuesta += ing.capitalize()+'. '
                            respuesta += '\n' 
                    if Clientes[update.message.chat_id].ver_alergenos:
                        alergenos = df_aux.iloc[fila_i]['alergenos']
                        if len(alergenos) > 0:
                            respuesta += 'Alergenos: '
                            for al in alergenos:
                                respuesta += al.capitalize()+'. '
                            respuesta += '\n'   
                    
            respuesta += '-'*15+'\n'
        
    bot.sendMessage(chat_id=update.message.chat_id, text=respuesta)

In [16]:
?MessageHandler

Object `MessageHandler` not found.


In [18]:
telegram.ext.RegexHandler?

Init signature:
telegram.ext.RegexHandler(
    pattern,
    callback,
    pass_groups=False,
    pass_groupdict=False,
    pass_update_queue=False,
    pass_job_queue=False,
    pass_user_data=False,
    pass_chat_data=False,
    allow_edited=False,
    message_updates=True,
    channel_post_updates=False,
    edited_updates=False,
)
Docstring:     
Handler class to handle Telegram updates based on a regex.

It uses a regular expression to check text messages. Read the documentation of the ``re``
module for more information. The ``re.match`` function is used to determine if an update should
be handled by this handler.

Attributes:
    pattern (:obj:`str` | :obj:`Pattern`): The regex pattern.
    callback (:obj:`callable`): The callback function for this handler.
    pass_groups (:obj:`bool`): Optional. Determines whether ``groups`` will be passed to the
        callback function.
    pass_groupdict (:obj:`bool`): Optional. Determines whether ``groupdict``. will be passed to
        the

Como añadir platos a la carta:

In [ ]:
carta.loc['Plato random',:] = [['cacahuetes'],'tapa',['cacahuetes'],1]

In [ ]:
carta

In [ ]:
def ver_alergenos(bot=mi_bot, update=mi_bot_updater):
    v = Clientes[update.message.chat_id].ver_alergenos
    Clientes[update.message.chat_id].ver_alergenos = not v
    bot.send_message(chat_id=update.message.chat_id, text='A partir de ahora'+' no'*v +\
                     ' se mostraran todos los alergenos de los platos.\n Puede revertir esta opción introduciendo nuevamente el comando /Alergenos')
    
def ver_ingredientes(bot=mi_bot, update=mi_bot_updater):
    v = Clientes[update.message.chat_id].ver_ingredientes
    Clientes[update.message.chat_id].ver_ingredientes = not v
    bot.send_message(chat_id=update.message.chat_id, text='A partir de ahora'+' no'*v +\
                     ' se mostraran todos los ingredientes de los platos.\n Puede revertir esta opción introduciendo nuevamente el comando /Ingredientes')

In [ ]:
def Añadir(bot=mi_bot, update=mi_bot_updater):
    Entendido = False #Por si nos dicen algo que no está en la lista
    
    mensaje = update.message.text
    mensaje = mensaje.lower() #Ponemos todo en minúsculas
#     print(update.message.date)
    print(update.message.from_user.full_name)
    for plato in carta.index:
        # Cantidad de veces que aparece el plato en el mensaje:
        c = len(re.findall(plato.lower(), mensaje))
        if c>0:
            Clientes[update.message.chat_id].AñadirPlato(plato, cantidad = c)
            Entendido = True
    
    if Entendido:
        bot.send_message(chat_id=update.message.chat_id, text='¿Desea añadir algo más?')
    else:
        bot.send_message(chat_id=update.message.chat_id, text='Lo sentimos, el plato seleccionado no está disponible en nuestro restaurante.')

In [ ]:
def Cesta(bot=mi_bot, update=mi_bot_updater):
    Clientes[update.message.chat_id].PrintSeleccion()

In [ ]:
mi_bot_updater.stop()
dispatcher.handlers = {0: []}
Clientes = {}

In [ ]:
#Definimos para cada comando la función que atenderá la peticion
#start_handler = CommandHandler('start', start)
ayuda_handler = CommandHandler('?', ayuda)
ayuda_handler = CommandHandler('Ayuda', ayuda)
#Salir_handler = CommandHandler('Salir', Salir)
RegistrarCliente_handler = CommandHandler('Registrar', RegistrarCliente)
#MostrarCarta_handler = CommandHandler('Carta', MostrarCarta)
MostrarCarta_handler = telegram.ext.RegexHandler('([C|c]arta)|([T|t]apa)|([E|e]ntrante)|([P|p]rimero)|([S|s]egundo)|([P|p]ostre)|([B|b]ebida)',MostrarCarta)

Ingredientes_handler = CommandHandler('Ingredientes', ver_ingredientes)
Alergenos_handler = CommandHandler('Alergenos', ver_alergenos)

#Añadir_handler = telegram.ext.RegexHandler('.*[A|a]ñadir',Añadir)
Añadir_handler = telegram.ext.RegexHandler('(.*[A|a](ñad))|(.*(?<![N|n]o\s)[Q|q]uiero)',Añadir)
#Quitar_handler = telegram.ext.RegexHandler('(.*[Q|q](uit))|(.*(?<=[N|n]o\s)[Q|q]uiero)',Quitar)

Cesta_handler = CommandHandler('Cesta', Cesta)
Yo_handler = CommandHandler('Yo', yo)
Eliminar_handler = CommandHandler('Eliminarme', DarDeBaja)

In [ ]:
dispatcher = mi_bot_updater.dispatcher

#dispatcher.add_handler(NuevoCliente_handler)
#dispatcher.add_handler(start_handler)
#dispatcher.add_handler(Salir_handler)
dispatcher.add_handler(ayuda_handler)
dispatcher.add_handler(RegistrarCliente_handler)
dispatcher.add_handler(MostrarCarta_handler)

dispatcher.add_handler(Ingredientes_handler)
dispatcher.add_handler(Alergenos_handler)
dispatcher.add_handler(Añadir_handler)
#dispatcher.add_handler(Quitar_handler)

dispatcher.add_handler(Cesta_handler)
dispatcher.add_handler(Yo_handler)
dispatcher.add_handler(Eliminar_handler)

In [ ]:
mi_bot_updater.start_polling()

Todo lo que se le puede aplicar a un update.message

['ATTACHMENT_TYPES', 'MESSAGE_TYPES', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_effective_attachment', '_id_attrs', '_parse_html', '_parse_markdown', '_quote', 'audio', 'author_signature', 'bot', 'caption', 'caption_entities', 'caption_html', 'caption_html_urled', 'caption_markdown', 'caption_markdown_urled', 'channel_chat_created', 'chat', 'chat_id', 'connected_website', 'contact', 'date', 'de_json', 'delete', 'delete_chat_photo', 'document', 'edit_caption', 'edit_date', 'edit_reply_markup', 'edit_text', 'effective_attachment', 'entities', 'forward', 'forward_date', 'forward_from', 'forward_from_chat', 'forward_from_message_id', 'forward_signature', 'from_user', 'game', 'group_chat_created', 'invoice', 'left_chat_member', 'location', 'media_group_id', 'message_id', 'migrate_from_chat_id', 'migrate_to_chat_id', 'new_chat_members', 'new_chat_photo', 'new_chat_title', 'parse_caption_entities', 'parse_caption_entity', 'parse_entities', 'parse_entity', 'photo', 'pinned_message', 'reply_audio', 'reply_contact', 'reply_document', 'reply_html', 'reply_location', 'reply_markdown', 'reply_media_group', 'reply_photo', 'reply_sticker', 'reply_text', 'reply_to_message', 'reply_venue', 'reply_video', 'reply_video_note', 'reply_voice', 'sticker', 'successful_payment', 'supergroup_chat_created', 'text', 'text_html', 'text_html_urled', 'text_markdown', 'text_markdown_urled', 'to_dict', 'to_json', 'venue', 'video', 'video_note', 'voice']

Todo lo que se le puede aplicar a update.message.from_user

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_id_attrs', 'bot', 'de_json', 'de_list', 'first_name', 'full_name', 'get_profile_photos', 'id', 'is_bot', 'language_code', 'last_name', 'mention_html', 'mention_markdown', 'name', 'send_audio', 'send_document', 'send_message', 'send_photo', 'send_sticker', 'send_video', 'send_video_note', 'send_voice', 'to_dict', 'to_json', 'username']

Proximos objetivos:
- **/yo: Muestra los datos del cliente**  
- **La función de pedir tiene que responter a: Añadir/Añade/Quiero(Quiero solo al principio)**  
- **Una función de quitar cosas de la cesta que responda a 'No quiero'/Quita/Quitar/Quitame**  
- **Una función que responda a mensajes: muestrame la carta, muestrame los primeros,.... regex -->[carta|tapas|primeros]... ---> if carta in mensaje muestra la carta, else ---> for cosa in [tapa,postre,....]: if cosa in mensaje....** 

- Poner una función que tramite el pedido en la que se muestre la cesta y un par de botones de confirmación
- Poner un TUTORIAL!!! --> al final de todas las funciones poner un if cliente.tutorial: aser cosas nasis
- **Poner una ayuda como dios manda.**  
- **/Comandos: Lista con todos los comandos**

In [ ]:
def yo(bot=mi_bot, update=mi_bot_updater):
    if update.message.chat_id in Clientes.keys():
        # Tomamos el cliente
        cliente = Clientes[update.message.chat_id]
        platos = ''
        if len(cliente.platos_elegidos)>0:
            for i in platos_elegidos:
                platos += i+'. '
        respuesta = 'Los siguientes datos son los datos almacenados en nuestro registro:'+'\n'+\
        '-Id de la conversación:'+str(update.message.chat_id)+'. Para poder contestar a las peticiones.'+'\n'+\
        '-Listado de platos escogidos:'+platos+'\n'+\
        '-El valor actual del pedido: '+str(cliente.precio)+'€\n'+\
        '-Si desea ver o no los ingredientes y los alergenos de los platos.'+'\n'+\
        '-Su nombre para tener un responsable del encargo: '+cliente.nombre
        
        bot.send_message(chat_id=update.message.chat_id, text=respuesta)
    else:
        bot.send_message(chat_id=update.message.chat_id, text='No se encuentra registrado en este servicio.')

In [ ]:
def DarDeBaja(bot=mi_bot, update=mi_bot_updater):
    IdCliente = update.message.chat_id
    if IdCliente in Clientes.keys():
        del Clientes[IdCliente]
        bot.send_message(chat_id=update.message.chat_id, text='Se le ha eliminado de nuestra base de datos.')
    else:
        bot.send_message(chat_id=update.message.chat_id, text='No se encuentra registrado en nuestra base de datos.')

In [ ]:
re.findall('(?<!No\s)quiero', 'quiero cocacola flan jam laad')

Bienvenido a la pagina de peticiones del restaurante Tan Rou Vao:  

Si en algún momento no respodemos a sus mensajes es porque el restaurante se encuentra cerrado.

Si es la primera vez que utiliza esta aplicación puede ver el listado de comandos mediante /Ayuda ó /?.
    
*Nota*: Esta aplicación no está binculada a ningún restaurante, solo es una tarea evaluable para una asignatura.

Bienvenido a la pagina de peticiones del restaurante Tan Rou Vao: Simplemente con un par de clicks y unos cuantos mensajes será capaz de encargar su pedido de manera facil y sencilla. 

Si en algún momento no recibe respuesta a sus mensajes, puede ser debido a que no se encuentra registrado en nuestra base de datos ó el restaurante se encuentra cerrado.

Si es la primera vez que utiliza esta aplicación puede ver el listado de comandos mediante /Ayuda ó /?.

*Nota*: Esta aplicación no está binculada a ningún restaurante, solo es una tarea evaluable para una asignatura.

In [ ]:
len('Bienvenido a la pagina de peticiones del restaurante Tan Rou Vao: Simplemente con unos cuantos mensajes será capaz de encargar su pedido de manera facil y sencilla.Si en algún momento no recibe respuesta a sus mensajes, puede ser debido a que no se encuentra registrado en nuestra base de datos ó el restaurante se encuentra cerrado.Si es la primera vez que utiliza esta aplicación puede ver el listado de comandos mediante /Ayuda ó /?.Nota: Esta aplicación no está binculada a ningún restaurante, solo es una tarea evaluable para una asignatura.')

Navegue por nuestra carta y realice los pedidos sin necesidad de venir al restaurante. Simplemente con un par de clicks y unos cuantos mensajes será capaz de encargar su pedido de manera facil y sencilla.

Realice pedidos sin necesidad de venir a nuestro restaurante. Simplemente con unos pocos mensajes hará su encargo.

- Introducción: 
    Bienvenido a la pagina de peticiones del restaurante Tan Rou Vao. Si en algún momento no respodemos a sus mensajes es porque el restaurante se encuentra cerrado.
    Dentro de esta aplicación podrá explorar y descubrir nuestra gran variedad de platos. Dando la posibilidad de detallar en todo momento los ingredientes y\o alergenos que contiene cada plato si el cliente lo desea.  
    Para poder disfrutar por completo de nuestra aplicación nuestros clientes se tienen que /Registrar. 
    Si es la primera vez que utiliza esta aplicación puede ver el listado de comandos mediante /Ayuda ó /?.
    
    *Nota*: Esta aplicación no está vinculada a ningún restaurante, solo es una tarea evaluable para una asignatura.
    
    
- Ayuda:
    /Ayuda ó /?: Muestran la ayuda de la aplicacción.  
    /Comandos: Muestra el listado de comandos.
    /Alergenos e /Ingredientes: Opción que regula si se quieren ver los Alergenos y/o Ingredientes de los platos.  
    /Registrar: Lleva a cabo el proceso de registro. Para este proceso solo registramos el Identificador del chat y le asociamos su nombre. Esto es para poder tramitar los pedidos y tener un usuario al que asignar el pedido.
    /Yo: Muestra los datos que tenemos almacenados sobre usted.
    /Eliminarme: Elimina su registro de la base de datos. Tras esta acción se tendría que volver a /Registrar si desea continuar utilizando la aplicación.
    
    /Carta: Muestra la carta de productos disponibles en nuestro restaurante. También se puede mostrar la carta con instrucciones como: "Ver la carta", "Muestrame la carta", etc. Y además, si está interesado en un tipo concreto de platos, como por ejemplo, los postres, se puede introducir "Muestrame los postres", "¿Qué postres tienes?", etc.
    
    ¿Y como se piden platos?
    Esto es muy sencillo, ya que solo hay que pedirlos de la siguiente forma "Quiero un Flan", "Añade un Flan", "Creo que quiero tomar un flan", etc. Si por algún casual, se desearía tomar dos flanes habría que introducir "Quiero un flan, un flan,...." ya que el programa solo reconoce los platos de uno en uno.
    
    ¿Y si pido algo que no quiero?
    En este caso, solo habría que escribir "No quiero\Quitame Nombre del plato a quitar", y al igual que en el caso de añadir platos también hay que introducirlos de uno en uno.
    
    /Cesta: Muestra la selección de platos que el cliente desea comprar.
    /Cuenta: Confirma que quiere encargar todos los platos incluidos en la /Cesta y tramita el pedido.
    
    /Tutorial: Si se encuentra registrado(/Registrar) puede iniciar el tutorial que le guiará en un ejemplo sencillo del uso de la aplicación.

# Versión 2

In [ ]:
import telebot
import time # Librería para hacer que el programa que controla el bot no se acabe.
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
import re
import numpy as np
TOKEN = '465196210:AAEd0NUrSnMD4ugh23SpQupeC3sZBXjZECU' # Nuestro token del bot.

In [ ]:
#Creamos nuestra instancia "mi_bot" a partir de ese TOKEN
mi_bot = telegram.Bot(token=TOKEN)
mi_bot_updater = Updater(mi_bot.token)

Y las funciones

In [ ]:
Clientes = {}

def Start(bot=mi_bot, update=mi_bot_updater):
    
    ClienteId = update.message.chat_id
    if update.message.chat_id not in Clientes.keys():
        #Introducimos el nombre
        Nuevo_Cliente = Cliente(bot,update,nombre = update.message.from_user.full_name, fecha_ingreso = update.message.date)
        Clientes[update.message.chat_id] = Nuevo_Cliente
        #bot.sendMessage(chat_id=update.message.chat_id, text="Se le ha registrado con exito.")
    
    mensaje = 'Bienvenido al chat de encargos para llevar del restaurante Tan Rou Vao.'+'\n'+\
    '\n'+\
    'Se ha registrado con éxito en nuestra aplicación. Recuerde que puede visualizar la carta mediante /Carta.'+'\n'+\
    '\n'+\
    'Se puede ver el listado de comandos mediante /Ayuda o /?. También hay disponible un pequeño listado de ejemplos accesible mediante /Ejemplos.'+'\n'+\
    '\n'+\
    'Disfrute de la aplicación.'
    
    bot.sendMessage(chat_id=update.message.chat_id, text=mensaje,
                     parse_mode=telegram.ParseMode.MARKDOWN)

In [ ]:
def ayuda(bot=mi_bot, update=mi_bot_updater):
    Ayuda = '    /Ayuda ó /?: Muestran la ayuda de la aplicacción.'+'\n'+\
    '/Comandos: Muestra un resumen de los comandos.'+'\n'+\
    '/Alergenos e /Ingredientes: Opción que regula si se quieren ver los Alergenos y/o Ingredientes de los platos.'+'\n'+\
    '/Registrar: Lleva a cabo el proceso de registro. Para este proceso solo registramos el Identificador del chat y le asociamos su nombre. Esto es para poder tramitar los pedidos y poder tener un usuario al que asignar el pedido.'+'\n'+\
    '/Yo: Muestra los datos que tenemos almacenados sobre usted.'+'\n'+\
    '/Eliminarme: Elimina su registro de la base de datos. Tras esta acción se tendría que volver a /Registrar si desea continuar utilizando la aplicación.'+'\n'+\
    '\n'+\
    '/Carta: Muestra la carta de productos disponibles en nuestro restaurante.'+'\n'+\
    '/Tapas,/Bebidas,/Entrantes,/Primeros,/Segundos,/Postres: Muestra los platos incluidos dentro de cada categoría.'+'\n'+\
    'También se puede mostrar la carta con instrucciones como: \"Ver la carta\", \"Muestrame la carta\", etc. Y además, si está interesado en un tipo concreto de platos, como por ejemplo, los postres, se puede introducir "Muestrame los postres", "¿Qué postres tienes?", etc.'+'\n'+\
    '\n'+\
    '¿Y como se piden platos?'+'\n'+\
    'Esto es muy sencillo, ya que solo hay que pedirlos de la siguiente forma \"Quiero un Flan\", \"Añade un Flan\", \"Creo que quiero tomar un flan\", etc. Si por algún casual, se desearía tomar dos flanes habría que introducir el nombre del producto dos veces(\"Quiero un flan, un flan,....\") ya que de momento el programa solo reconoce los platos de uno en uno.'+'\n'+\
    '\n'+\
    '¿Y si pido algo que no quiero?'+'\n'+\
    'En este caso, solo habría que escribir \"No quiero\Quitame Nombre del plato a quitar\", y al igual que en el caso de añadir platos también hay que introducirlos de uno en uno.'+'\n'+\
    '\n'+\
    '/Cesta: Muestra la selección de platos que el cliente desea comprar.'+'\n'+\
    '/Cuenta: Confirma que quiere encargar todos los platos incluidos en la /Cesta y tramita el pedido.'+'\n'+\
    '\n'+\
    '/Tutorial: Si se encuentra registrado(/Registrar) puede iniciar el tutorial que le guiará en un ejemplo sencillo del uso de la aplicación.'
    bot.sendMessage(chat_id=update.message.chat_id, text=Ayuda)
    
def ayuda(bot=mi_bot, update=mi_bot_updater):
    ayuda ='-'*30+'\n'+\
    '*Básicos:*'+'\n'+\
    '    /?'+'\n'+\
    '    /Ayuda: Muestra esta lista de comandos.'+'\n'+\
    '    /Registrar: Se registra en la aplicación y le da acceso a todos los comandos.'+'\n'+\
    '    /Yo: Muestra la información que tenemos sobre usted.'+'\n'+\
    '    /Ejemplos: Muestra una pequeña lista de ejemplos.'+'\n'+\
    '    /Pista: Muestra un ejemplo aleatorio.'+'\n'+\
    '    /Eliminarme: Elimina su registro a la base de datos.'+'\n'+\
    '    /start: Inicia el bot y le registra(Si no se encucentra registrado).'+'\n'+\
    '-'*30+'\n'+\
    '*Alérgenos e Ingredientes:*'+'\n'+\
    '    /Alergenos: Ver\\No ver los Alérgenos de los platos.'+'\n'+\
    '    /Ingredientes: Ver\\No ver los Ingredientes de los platos.'+'\n'+\
    '-'*30+'\n'+\
    '*Relacionados con la carta:*'+'\n'+\
    '    /Carta: Muestra la carta.'+'\n'+\
    '    /Tapas: Muestra la sección de Tapas.'+'\n'+\
    '    /Bebidas: Muestra la sección de Bebidas.'+'\n'+\
    '    /Entrantes: Muestra la sección de Entrantes.'+'\n'+\
    '    /Primeros: Muestra la sección de Primeros.'+'\n'+\
    '    /Segundos: Muestra la sección de Segundos.'+'\n'+\
    '    /Postres: Muestra la sección de Postres.'+'\n'+\
    '    *Nota*: Todos estos comandos se pueden introducir en la misma instrucción: Muestrame las bebidas, las tapas y los postres.'+'\n'+\
    '-'*30+'\n'+\
    '*Reecomendaciones y fotos:*'+'\n'+\
    '    /Foto de "nombre de plato": Muestra una foto del plato nombrado.'+'\n'+\
    '    /Recomendar: Muestra un plato aleatorio de la carta.'+'\n'+\
    '    /Recomendar plato con "Ingrediente": Muestra un plato aleatorio de la carta que contenga ese ingrediente.'+'\n'+\
    '-'*30+'\n'+\
    '*Relacionados con los pedidos:*'+'\n'+\
    '    /Cesta: Muestra los platos seleccionados.'+'\n'+\
    '    /Cuenta: Tramita el pedido y le pasamos la Cuenta.'+'\n'+\
    '    /Quiero, /Añadir, /Añade \"Nombre del plato\".'+'\n'+\
    '    /Quita, No quiero, /Quitame \"Nombre del plato\".'+'\n'+\
    '-'*30
    
    bot.sendMessage(chat_id=update.message.chat_id, text=ayuda,
                        parse_mode=telegram.ParseMode.MARKDOWN)

In [ ]:
class Cliente:
    def __init__(self, bot, update, nombre, fecha_ingreso):
        self.platos_elegidos = {}
        self.precio = 0
        
        # Variable que cuenta el número de veces que este
        #    cliente ha realizado un encargo y no ha ido a 
        #    recogerlo.
        self.nivel_impagos = 0
        self.ver_alergenos = False
        self.ver_ingredientes = False
        self.nombre = nombre
        self.fecha_ingreso = fecha_ingreso
        #Guardamos el id del mensaje para luedo asegurarnos de que no ha pedido nada más
        self.Id_mensage_compra = None
        
        self.bot = bot
        self.update = update
        
    def PrintSeleccion(self):
        if len(self.platos_elegidos)==0:
            mensaje = 'No ha elegido platos. \n'
        else:
            mensaje = '*Platos elegidos:*\n'
            for plato in self.platos_elegidos.keys():
                mensaje += ' -'+plato.capitalize()+' x'+str(self.platos_elegidos[plato])+': '+\
                str(self.platos_elegidos[plato]*carta.loc[plato,'precio'])+'€\n'
                
                if self.ver_ingredientes:
                    Ingredientes = carta.loc[plato,'ingredientes']
                    if len(Ingredientes)> 0:
                        mensaje += '    -*Ingredientes:* '
                        for ing in Ingredientes:
                            mensaje += ing.capitalize() +'. '
                        mensaje += '\n'
                    
                if self.ver_alergenos:
                    Alergenos = carta.loc[plato,'alergenos']
                    if len(Alergenos) > 0:
                        mensaje += '    -*Alérgenos:* '
                        for al in Alergenos:
                            mensaje += al.capitalize() +'. '
                        mensaje += '\n'
                    
                
                    
            mensaje += '-'*20+'\n'
            mensaje += '*Total:* '+str(self.precio)+' €'
        # Enviamos la respuesta
        self.bot.sendMessage(chat_id=self.update.message.chat_id, text=mensaje,
                        parse_mode=telegram.ParseMode.MARKDOWN)
        
    def AñadirPlato(self,plato,cantidad):
        # plato --> Plato elegido
        # cantidad --> Veces que se ha pedido
        if plato in self.platos_elegidos.keys():
            self.platos_elegidos[plato] += cantidad
        else:
            self.platos_elegidos[plato] = cantidad #Solo lo ha pedido una vez
        #Actualizamos el precio del pedido
        self.precio += cantidad*carta.loc[plato,'precio']
        
        if cantidad ==1:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, 
                                 text='Se ha añadido a la cesta una unidad de '+plato.capitalize()+'.')
        else:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, 
                                 text='Se han añadido a la cesta '+str(cantidad)+' unidades de '+plato.capitalize()+'.')
    
    def QuitarPlato(self,plato,cantidad):
        # plato --> Plato a quitar
        # cantidad --> Veces que se quiere quitar
        self.platos_elegidos[plato] -= cantidad
        # Si se quiere quitar más veces de las que aparece:
        if self.platos_elegidos[plato] <= 0:
            # Si es negativo nos quedamos con el número de platos que tenía seleccionados
            num_platos = cantidad+self.platos_elegidos[plato]
            self.precio -= num_platos*carta.loc[plato,'precio']
            del self.platos_elegidos[plato]
        else:
            num_platos = cantidad
            #Actualizamos el precio del pedido asegurandonos de que no le salga rentable poner y quitar platos
            self.precio -= cantidad*carta.loc[plato,'precio']
        
        if num_platos ==1:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, 
                                 text='Se ha quitado de la cesta una unidad de '+plato.capitalize()+'.')
        else:
            self.bot.sendMessage(chat_id=self.update.message.chat_id, 
                                 text='Se han eliminado '+str(num_platos)+' unidades de '+plato.capitalize()+' de su cesta.')
    
    def Comprar(self,fecha):
        print('*'*50)
        if self.nivel_impagos == 1:
            print('¡¡¡¡Peligro!!!! Este cliente ha realizado',self.nivel_impagos,'pedido que nunca ha recogido')
        elif self.nivel_impagos > 1:
            print('¡¡¡¡Peligro!!!! Este cliente ha realizado',self.nivel_impagos,'pedidos que nunca ha recogido')
            
        print('Demandante del pedido:'+self.nombre)
        print('Importe total del pedido: '+str(self.precio)+' €')
        print('-'*25)
        print('Contenido del pedido:')
        for plato in self.platos_elegidos.keys():
            print(' -'+plato.capitalize()+' x'+str(self.platos_elegidos[plato])+': '+str(self.platos_elegidos[plato]*carta.loc[plato,'precio'])+'€')       
        # Este trozo de código es para notificar al propietario/recepcionista del restaurante la información del pedido:
        print('Fecha en la que se solicitó el pedido:'+str(fecha))
        print('*'*50)

In [ ]:
Clientes = {}
def RegistrarCliente(bot=mi_bot, update=mi_bot_updater):
    ClienteId = update.message.chat_id
    if update.message.chat_id in Clientes.keys():
        bot.sendMessage(chat_id=update.message.chat_id, text="Ya se encuentra registrado. Recuerde que puede visualizar nuestra carta mediante /Carta.")
    else:
        #Introducimos el nombre
        Nuevo_Cliente = Cliente(bot,update,nombre = update.message.from_user.full_name, fecha_ingreso = update.message.date)
        Clientes[update.message.chat_id] = Nuevo_Cliente
        bot.sendMessage(chat_id=update.message.chat_id, text="Se le ha registrado con exito. Recuerde que puede visualizar nuestra carta mediante /Carta.")

Definimos la carta:
+ **nombre:** Columna del nombre del plato.  
+ **precio:** Columna del precio del plato.  
+ **categorias:** Columna de las categorías del plato: primero, segundo, postre,....  
+ **ingredientes:** Columna con los ingredientes que lleva el plato.  
+ **alergenos:** Alergenos que lleva el plato.  
+ **url:** url donde se encuentra la imagen del plato.

In [ ]:
import pandas as pd

carta = pd.DataFrame({'nombre' : ['Chuletas con patatas','queso frito','Paella Valenciana','Flan','Cocacola'],
                     'precio' : [13,4,5,2,1.2],
                     'categorias' : ['segundo','entrante','primero','postre','bebida'],
                     'ingredientes' : [['Chuletas','patatas'],
                                       ['Queso'],
                                       ['Arroz','verduras variadas','carne de pollo'],
                                       ['huevo','leche'],
                                       []],
                     'alergenos' : [[],
                                    ['lactosa'],
                                    ['gluten'],
                                    ['huevos','lactosa'],
                                    []],
                     'url' : ['https://www.rebanando.com/media/457516447hd-jpg_crop.jpeg/rh/chuletas-de-cerdo-al-horno.jpg',
                              'https://cocina-casera.com/wp-content/uploads/2017/11/queso-frito.jpg',
                              'https://www.hogarmania.com/archivos/201104/paella-valencia-v-848x477x80xX.jpg',
                              'http://www.annarecetasfaciles.com/files/flan-de-huevo-2a-650x366.jpg',
                             'http://www.elcorreogallego.es/img/noticias/20080302/coca_117996.jpg']})
carta = carta.set_index('nombre')
carta = carta.sort_index()
carta

Como añadir platos a la carta:

In [ ]:
#carta.loc['Plato random',['ingredientes','categorias','alergenos','precio']] = [['cacahuetes'],'tapa',['cacahuetes'],1]

In [ ]:
#[nombe,ingredientes,categoria,alergenos,precio]
platos = [
    ['cacahuetes',['cacahuetes'],'tapa',['cacahuetes'],1,'http://2.bp.blogspot.com/_sH2vxqWQ53U/S2VYrmlcwmI/AAAAAAAAAhs/CMVepeRtroc/s400/cacauets2A.JPG'],
    ['aceitunas',['aceitunas'],'tapa',[],1,'http://aceitunassalinas.com/wp-content/uploads/2015/09/aceituna-partida-jaen-600x450.jpg'],
    ['caracoles',['caracoles'],'tapa',['moluscos'],2.5,'http://restaurantcarmen.com/wp-content/uploads/2014/05/tapa-caracoles-restaurante-carmen.jpg'],
    ['pincho tortilla',['huevos','patatas'],'tapa',['huevos'],1.5,'https://media-cdn.tripadvisor.com/media/photo-s/09/24/d6/a9/restaurante-la-morenica.jpg'],
    ['Gambas al Ajillo',['Gambas','Ajo'],'entrante',['marisco'],3.5,'http://estoyhechouncocinillas.com/wp-content/uploads/2014/06/gambas_al_ajillo.png'],
    ['Sushi de salmón',['arroz','salmón'],'entrante',[],5.2,'https://cocinista-vsf.netdna-ssl.com/download/bancorecursos/recetas/sushi-sake.jpg'],
    ['Sushi de gambas',['arroz','gambas'],'entrante',['marisco'],4.5,'http://2.bp.blogspot.com/-m9dxbt47B90/UEWdt98I8jI/AAAAAAAAJTI/Xz5sNFtCJrQ/s1600/381+-+Sushi+de+gamba.jpg'],
    ['Pulpo',['Pulpo'],'entrante',['marisco'],8.5,'https://i.ytimg.com/vi/oWjmKhA6NE0/maxresdefault.jpg'],
    ['Parrillada de Verduras',['Pimientos','Patatas','Berenjenas'],'primero',[],9.5,'https://i.blogs.es/72da79/parrillada-verduras-perfecta/450_1000.jpg'],
    ['Bacalao al Pil-Pil',['Bacalao','pil-pil'],'primero',[],19.3,'http://factorgastronomic.com/wp-content/uploads/2015/01/bacalla-al-pil-pil.jpg'],
    ['Sopa de carne',['carne'],'primero',[],8.5,'https://2.bp.blogspot.com/-t_tqFPViLic/VPQ4NoPxDuI/AAAAAAAADqw/0Qr_AFwkt5I/s1600/Caldo%2Bde%2Bres.jpg'],
    ['Sopa de pescado',['pescado'],'primero',['pescado'],9.5,'https://unareceta.com/wp-content/uploads/2016/08/receta-sopa-de-pescado-con-patatas.jpg'],
    ['Ensalada japonesa',['algas variadas'],'primero',[],3.4,'https://t1.rg.ltmcdn.com/es/images/2/4/2/img_ensalada_japonesa_con_zanahoria_50242_600.jpg'],
    ['Huevos Rotos con Patatas',['Huevos','Patatas'],'primero',['huevos'],9.5,'http://3.bp.blogspot.com/-LaQrlL3BWqw/Ut0USYI-t4I/AAAAAAAANZI/ZL7EQmWyW0I/s1600/huevos_rotos_5.JPG'],
    ['Solomillo con patatas',['Solomillo','patatas'],'segundo',[],12.5,'https://img.recetascomidas.com/recetas/320_240/solomillo-con-patatas.jpg'],
    ['Entrecot con patatas',['Entrecot','patatas'],'segundo',[],15.2,'http://4.bp.blogspot.com/-pzdJhN_NclE/T-oTaHag2XI/AAAAAAAAB3w/3uxNQHG6UHU/s1600/ENTRECOR+1.JPG'],
    ['Merluza con patatas',['Merluza','patatas'],'segundo',['pescado'],16.7,'http://alegriaenlacocina.com/wp-content/uploads/2013/08/perca-empanada.jpg'],
    ['costillas barbacoa',['costillas de cerdo','salsa barbacoa'],'segundo',[],5.9,'https://canales.okdiario.com/recetas/img/2016/11/07/costillas-cocidas-al-horno-con-salsa-barbacoa-655x368.jpg'],
    ['Lomo con patatas',['lomo','patatas'],'segundo',[],5.1,'https://media-cdn.tripadvisor.com/media/photo-s/07/24/e0/d6/lomo-adosado-con-patatas.jpg'],
    ['Natillas',['Leche','huevo','canela','azucar'],'postre',['lactosa','huevos'],3.5,'https://canales.okdiario.com/recetas/img/2017/05/16/natillas-caseras-655x368.jpg'],
    ['Tarta de Queso',['queso','galletas'],'postre',['lactosa','gluten'],4.8,'https://www.dulcespostres.com/wp-content/uploads/2014/05/Tarta-de-queso-5.jpg'],
    ['Macedonia de frutas',['melocotón','melón','naranja','platano','manzana'],'postre',[],2,'https://cocina-casera.com/wp-content/uploads/2012/05/macedonia-frutas-almibar.jpg'],
    ['Helado',['chocolate','vainilla','coco','nata','fresa'],'postre',['lactosa'],2.4,'http://cdn2.cocinadelirante.com/sites/default/files/styles/gallerie/public/images/2016/07/helado.jpg'],
    ['fanta de naranja',[],'bebida',[],1.2,'http://valsegura.com/wp-content/uploads/2016/02/fanta-naranja-20cl.jpg'],
    ['fanta de limón',[],'bebida',[],1.2,'http://www.hielosahy.com/tienda/51-large_default/fanta-naranja-500ml-botella-de-plastico.jpg'],
    ['fanta de manzana',[],'bebida',[],1.8,'https://internacionalgourmet.com/9-large_default/refresco-fanta-sabores-a-manzana.jpg'],
    ['agua grande',[],'bebida',[],1,'https://www.compraonline.grupoeroski.com/images/152587.jpg'],
    ['agua pequeña',[],'bebida',[],0.75,'http://1.bp.blogspot.com/_T-Dt3CPWrJ8/TDAtKXW1CYI/AAAAAAAAAR8/1BGIwjPnfzw/s1600/AGUA+MINERAL+NATURAL.jpg'],
    ['café',[],'bebida',[],1.2,'http://static.t13.cl/images/sizes/1200x675/1440370450-150823123913cafesalpicarcafe624x351thinkstocknocredit.jpg']
]

for nombre,ingredientes,categoria,alergenos,precio,url in platos:
    carta.loc[nombre,['ingredientes','categorias','alergenos','precio','url']] = [ingredientes,categoria,alergenos,precio,url]

In [ ]:
carta

In [ ]:
def MostrarCarta(bot=mi_bot, update=mi_bot_updater):
    respuesta = ''
    for categoria in ['tapa', 'bebida','entrante', 'primero', 'segundo', 'postre']:
        # Definimos el mensaje de la carta
        mensaje = update.message.text
        mensaje = mensaje.lower() #Ponemos todo en minúsculas
        if 'carta' in mensaje or categoria in mensaje: #Apaño para que salga todo cuando se encuentren la palabra 'carta' 
                                                       #y la categoría en el mismo mensage
            respuesta += '*'+categoria.capitalize()+'*:\n'
            df_aux = carta.loc[carta['categorias'] == categoria,:]
            
            if df_aux.shape[0] > 0:
                for fila_i in range(df_aux.shape[0]):
                    respuesta += ' +'+df_aux.index[fila_i].capitalize()+': '+ str(df_aux.iloc[fila_i]['precio'])+' €.\n'
                    
                    if Clientes[update.message.chat_id].ver_ingredientes:
                        ingredientes = df_aux.iloc[fila_i]['ingredientes']
                        if len(ingredientes) > 0:
                            respuesta += '    -*Ingredientes:* '
                            for ing in ingredientes:
                                respuesta += ing.capitalize()+'. '
                            respuesta += '\n' 
                    if Clientes[update.message.chat_id].ver_alergenos:
                        alergenos = df_aux.iloc[fila_i]['alergenos']
                        if len(alergenos) > 0:
                            respuesta += '    -*Alérgenos:* '
                            for al in alergenos:
                                respuesta += al.capitalize()+'. '
                            respuesta += '\n'   
                    
            respuesta += '-'*15+'\n'
        
    bot.sendMessage(chat_id=update.message.chat_id, text=respuesta,
                        parse_mode=telegram.ParseMode.MARKDOWN)

In [ ]:
def ver_alergenos(bot=mi_bot, update=mi_bot_updater):
    v = Clientes[update.message.chat_id].ver_alergenos
    Clientes[update.message.chat_id].ver_alergenos = not v
    bot.send_message(chat_id=update.message.chat_id, text='A partir de ahora'+' *no*'*v +\
                     ' se mostraran todos los alérgenos de los platos.\n Puede revertir esta opción introduciendo nuevamente el comando /Alergenos.',
                        parse_mode=telegram.ParseMode.MARKDOWN)
    
def ver_ingredientes(bot=mi_bot, update=mi_bot_updater):
    v = Clientes[update.message.chat_id].ver_ingredientes
    Clientes[update.message.chat_id].ver_ingredientes = not v
    bot.send_message(chat_id=update.message.chat_id, text='A partir de ahora'+' *no*'*v +\
                     ' se mostraran todos los ingredientes de los platos.\n Puede revertir esta opción introduciendo nuevamente el comando /Ingredientes.',
                        parse_mode=telegram.ParseMode.MARKDOWN)

In [ ]:
def Añadir(bot=mi_bot, update=mi_bot_updater):
    Entendido = False #Por si nos dicen algo que no está en la lista
    
    mensaje = update.message.text
    mensaje = mensaje.lower() #Ponemos todo en minúsculas
    for plato in carta.index:
        # Cantidad de veces que aparece el plato en el mensaje:
        c = len(re.findall(plato.lower(), mensaje))
        if c>0:
            Clientes[update.message.chat_id].AñadirPlato(plato, cantidad = c)
            Entendido = True
    
    if Entendido:
        bot.send_message(chat_id=update.message.chat_id, text='¿Desea añadir algo más?')
    else:
        bot.send_message(chat_id=update.message.chat_id, text='Lo sentimos, el plato seleccionado no está disponible en nuestro restaurante.')

In [ ]:
def Quitar(bot=mi_bot, update=mi_bot_updater):
    Entendido = False #Por si nos dicen algo que no está en la lista
    
    mensaje = update.message.text
    mensaje = mensaje.lower() #Ponemos todo en minúsculas
    
    cliente = Clientes[update.message.chat_id]
    for plato in cliente.platos_elegidos.keys():
        # Cantidad de veces que aparece el plato en el mensaje:
        c = len(re.findall(plato.lower(), mensaje))
        if c>0:
            Clientes[update.message.chat_id].QuitarPlato(plato, cantidad = c)
            Entendido = True
    
    if not Entendido:
        bot.send_message(chat_id=update.message.chat_id, text='El plato seleccionado no se encuentra en su cesta.')

In [ ]:
def Cesta(bot=mi_bot, update=mi_bot_updater):
    if update.message.chat_id in Clientes.keys():
        Clientes[update.message.chat_id].PrintSeleccion()
    else:
        bot.send_message(chat_id=update.message.chat_id, 
                         text='Aún no le hemos asignado una cesta. Para asignarle una es necesario que se registre(/Registrar).')

In [ ]:
def yo(bot=mi_bot, update=mi_bot_updater):
    if update.message.chat_id in Clientes.keys():
        # Tomamos el cliente
        cliente = Clientes[update.message.chat_id]
        platos = ''
        if len(cliente.platos_elegidos)>0:
            for i in cliente.platos_elegidos:
                platos += i+'. '
        respuesta = 'Los siguientes datos son los datos almacenados en nuestro registro:'+'\n'+\
        '    -*Id de la conversación:*'+str(update.message.chat_id)+'\n'+\
        '    -*Listado de platos seleccionados:*'+platos+'\n'+\
        '    -*Valor actual del pedido:* '+str(cliente.precio)+'€\n'+\
        '    -*Si desea ver o no los ingredientes y los alérgenos de los platos.*'+'\n'+\
        '    -*Nombre:* '+cliente.nombre
        
        bot.send_message(chat_id=update.message.chat_id, text=respuesta,
                         parse_mode=telegram.ParseMode.MARKDOWN)
    else:
        bot.send_message(chat_id=update.message.chat_id, 
                         text='No se encuentra registrado en este servicio. Se puede registrar mediante /Registrar.')

In [ ]:
def DarDeBaja(bot=mi_bot, update=mi_bot_updater):
    IdCliente = update.message.chat_id
    if IdCliente in Clientes.keys():
        cliente = Clientes[update.message.chat_id]
        print('\n'+ '*'*50)
        print('\"'+cliente.nombre+'\" registrado en '+ str(cliente.fecha_ingreso) +' se ha dado de baja de nuestros servicios en \"'+str(update.message.date)+'\".')
        try:
            print('Duración: ',update.message.date - cliente.fecha_ingreso,sep = '')
        except:
            print('Error de fechas.')
        print('*'*50)
        del Clientes[IdCliente]
        bot.send_message(chat_id=update.message.chat_id, text='Se le ha eliminado de nuestra base de datos. Para volver a registrarse recuerde que podía hacerlo mediante /Registrarse o volviendo a iniciar el bot mediante /start.')
    else:
        bot.send_message(chat_id=update.message.chat_id, text='No se encuentra registrado en nuestra base de datos.')

In [ ]:
def Cuenta(bot=mi_bot, update=mi_bot_updater):
    
    if update.message.chat_id in Clientes.keys():
        
        if len(Clientes[update.message.chat_id].platos_elegidos)==0:
            mensaje = 'No ha elegido platos. \n'
            bot.send_message(chat_id=update.message.chat_id,text=mensaje)
        else:
            #Guardamos el id del mensaje para luedo asegurarnos de que no ha pedido nada más
            Clientes[update.message.chat_id].Id_mensage_compra = update.message.message_id
            
            location_keyboard = telegram.KeyboardButton(text="Sí. Finalizar pedido.")
            contact_keyboard = telegram.KeyboardButton(text="No. Seguir pidiendo.")
            custom_keyboard = [[ location_keyboard, contact_keyboard ]]
            reply_markup = telegram.ReplyKeyboardMarkup(custom_keyboard)
            
            mensaje = '*Su pedido contiene los siguientes platos:*\n'
            for plato in Clientes[update.message.chat_id].platos_elegidos.keys():
                mensaje += ' -'+plato.capitalize()+' x'+str(Clientes[update.message.chat_id].platos_elegidos[plato])+': '+\
                str(Clientes[update.message.chat_id].platos_elegidos[plato]*carta.loc[plato,'precio'])+'€\n'
                
                if Clientes[update.message.chat_id].ver_ingredientes:
                    Ingredientes = carta.loc[plato,'ingredientes']
                    if len(Ingredientes)> 0:
                        mensaje += '    -*Ingredientes:* '
                        for ing in Ingredientes:
                            mensaje += ing.capitalize() +'. '
                        mensaje += '\n'
                    
                if Clientes[update.message.chat_id].ver_alergenos:
                    Alergenos = carta.loc[plato,'alergenos']
                    if len(Alergenos) > 0:
                        mensaje += '    -*Alérgenos:* '
                        for al in Alergenos:
                            mensaje += al.capitalize() +'. '
                        mensaje += '\n'
                    
                
                    
            mensaje += '-'*20+'\n'
            mensaje += "*Importe Total:* " +str(Clientes[update.message.chat_id].precio)+' €'
            
            mensaje += '\n'+'='*30+'\n'
            mensaje += '*¿Todos los datos son correctos?*'
            bot.send_message(chat_id=update.message.chat_id, 
                             text=mensaje, 
                             reply_markup=reply_markup,
                             parse_mode=telegram.ParseMode.MARKDOWN)
    else:
        bot.send_message(chat_id=update.message.chat_id, 
                         text="No podelos tramitar su pedido si *no* se encuentra registrado.",
                        parse_mode=telegram.ParseMode.MARKDOWN)
        

In [ ]:
def SeguirComprando(bot=mi_bot, update=mi_bot_updater):
    Clientes[update.message.chat_id].Id_mensage_compra = None
    reply_markup = telegram.ReplyKeyboardRemove()
    bot.send_message(chat_id=update.message.chat_id, text="Recuerde que puede mirar la carta con /Carta.",
                     reply_markup=reply_markup)

In [ ]:
def TramitarCompra(bot=mi_bot, update=mi_bot_updater):
    reply_markup = telegram.ReplyKeyboardRemove()
    #Si el último mensaje que ha mandao no ha sio el de /Cuenta #
    if update.message.message_id != Clientes[update.message.chat_id].Id_mensage_compra+2:
        Clientes[update.message.chat_id].Id_mensage_compra = None
        bot.send_message(chat_id=update.message.chat_id, 
                         text="Ha ocurrido un error durante la tramitación. Por favor, vuelva a intentar solicitar la /Cuenta.",
                         reply_markup=reply_markup)
    else:
        #Cliente[update.message.chat_id].Comprar(fecha = update.message.date)
        print('\n'+'*'*50)
        cliente = Clientes[update.message.chat_id]
        print('Demandante del pedido:'+cliente.nombre)
        print('Importe total del pedido: '+str(cliente.precio)+' €')
        print('-'*25)
        print('Contenido del pedido:')
        for plato in cliente.platos_elegidos.keys():
            print(' -'+plato.capitalize()+' x'+str(cliente.platos_elegidos[plato])+': '+str(cliente.platos_elegidos[plato]*carta.loc[plato,'precio'])+'€')       
        print('-'*25)
        # Este trozo de código es para notificar al propietario/recepcionista del restaurante la información del pedido:
        print('Fecha en la que se solicitó el pedido:'+str(update.message.date))
        print('*'*50+'\n')
        
        #Le eliminamos la selección realizada para que no pida lo mismo por accidente
        cliente.precio = 0
        cliente.platos_elegidos = {}
        
        bot.send_message(chat_id=update.message.chat_id, 
                         text="Su encargo será atendido en breves. Gracias por utilizar el servicio telemático de nuestro restaurante.",
                         reply_markup=reply_markup)

In [ ]:
def Ejemplos(bot=mi_bot, update=mi_bot_updater):
    ejemplos = '*Ejemplos de posibles comandos:*'+'\n'+\
    '\n'+\
    '-/Alergenos: Se mostrarán los alergenos de los platos.'+'\n'+\
    '\n'+\
    '-/Ingredientes: Se mostrarán los ingredientes de los platos.'+'\n'+\
    '\n'+\
    '_-Quiero un flan:_ Añadirá un flan a la cesta.'+'\n'+\
    '\n'+\
    '_-¿Me puedes añadir una cocacola y un flan?:_ Añadirá otro flan y una cocacola a la cesta.'+'\n'+\
    '\n'+\
    '_-Quitame un flan:_ Quitará un flan de la lista.'+'\n'+\
    '\n'+\
    '_-No quiero la cocacola:_ Quitará una cocacola de la lista.'+'\n'+\
    '\n'+\
    '_-Muéstrame la carta:_ Aparecerá el resumen de la carta(Si /Ingredientes y\\o /Alergenos se encuentran activados estos también se mostrarán).'+'\n'+\
    '\n'+\
    '_-Muéstrame los primeros y los postres:_ Solo aparecerá las secciones de primeros y postres.'+'\n'+\
    '\n'+\
    '_-Quiero que me muestres los primeros:_ Se mostrará nuestra sección de primeros platos.'+'\n'+\
    '\n'+\
    '_-¿Qué hay de postre?:_ Muestra la sección de postres.'+'\n'+\
    '\n'+\
    '_-Mándame una foto de los cacahuetes:_ El sistema manda una foto de los cacahuetes.'+'\n'+\
    '\n'+\
    '_-Foto de las aceitunas:_ El sistema manda una foto de las aceitunas.'+'\n'+\
    '\n'+\
    '_-/Pista:_ El bot te manda una instrucción a modo de ejemplo.'+'\n'+\
    '\n'+\
    '_-Recomiendame algún plato:_ El bot manda una foto de un plato de la carta junto con su nombre.'+'\n'+\
    '\n'+\
    '_-Recomiendame un plato con patatas:_ El bot manda una foto de un plato de la carta que lleve patatas junto con su nombre.'
    
    bot.send_message(chat_id=update.message.chat_id, 
                     text=ejemplos,
                     parse_mode=telegram.ParseMode.MARKDOWN)

In [ ]:
def EnviarFoto(bot=mi_bot, update=mi_bot_updater):
    
    mensaje = update.message.text
    mensaje = mensaje.lower() #Ponemos todo en minúsculas
    
    for plato in carta.index:
        if plato.lower() in mensaje:
            # Enviamos la foto desde la url
            bot.send_photo(chat_id=update.message.chat_id, photo=carta.loc[plato,'url'],caption = 'Foto de '+plato.capitalize())

In [ ]:
def Pista(bot=mi_bot, update=mi_bot_updater):
    pistas = ['Muéstrame las bebidas, las tapas y los postres',
             'Quiero ver la carta',
             '/Alergenos porque quiero ver los platos que llevan gluten',
             'Quiero una cocacola y unos cacahuetes',
             'No quiero la cocacola',
             'Mándame la foto del flan',
             'Recomiéndame un plato que lleve queso.',
             'Muéstrame las bebidas',
             'Quiero un flan',
             '¿Me puedes añadir una cocacola y un flan?',
             'Quítame un flan',
             'Muéstrame la carta',
             'Muéstrame los primeros y los postres',
             'Quiero que me muestres los primeros',
             '¿Qué hay de postre?',
             'Mándame una foto de los cacahuetes',
             'Foto de las aceitunas',
             'Recomiéndame algún plato',
             'Recomiéndame un plato con patatas',
             '/Cesta para ver lo que llevo encargado']
    
    
    bot.send_message(chat_id=update.message.chat_id, 
                     text=np.random.choice(pistas))

In [ ]:
def Recomendar(bot=mi_bot, update=mi_bot_updater):
    # Platos que podemos elegir
    Platos_Posibles = []
    
    mensaje = update.message.text
    mensaje = mensaje.lower() #Ponemos todo en minúsculas
    
    for plato in carta.index:
        for ingrediente in carta.loc[plato,'ingredientes']:
            if ingrediente.lower() in mensaje:
                Platos_Posibles.append(plato)
   
    if len(Platos_Posibles)>0:
        plato = np.random.choice(Platos_Posibles)
        bot.send_photo(chat_id=update.message.chat_id, photo=carta.loc[plato,'url'],caption = 'Le recomendamos: '+plato.capitalize())
    else:
        plato = np.random.choice(carta.index)
        bot.send_photo(chat_id=update.message.chat_id, photo=carta.loc[plato,'url'],caption = 'Le recomendamos: '+plato.capitalize())

In [ ]:
#Definimos para cada comando la función que atenderá la peticion
start_handler = CommandHandler('Start', Start)

ayuda_handler = CommandHandler('?', ayuda)
ayuda2_handler = CommandHandler('Ayuda', ayuda)
Ejemplos_handler = CommandHandler('Ejemplos', Ejemplos)
Pista_handler = CommandHandler('Pista', Pista)

Recomendar_handler = telegram.ext.RegexHandler('(.*[R|r]ecom)',Recomendar)

RegistrarCliente_handler = CommandHandler('Registrar', RegistrarCliente)
MostrarCarta_handler = telegram.ext.RegexHandler('(.*[C|c]arta)|(.*[T|t]apa)|(.*[E|e]ntrante)|(.*[P|p]rimero)|(.*[S|s]egundo)|(.*[P|p]ostre)|(.*[B|b]ebida)',MostrarCarta)

Ingredientes_handler = CommandHandler('Ingredientes', ver_ingredientes)
Alergenos_handler = CommandHandler('Alergenos', ver_alergenos)

Foto_handler = telegram.ext.RegexHandler('(.*[F|f]oto)',EnviarFoto)

Añadir_handler = telegram.ext.RegexHandler('(.*[A|a](ñad))|(.*(?<![N|n]o\s)[Q|q]uiero)',Añadir)
Quitar_handler = telegram.ext.RegexHandler('(.*[Q|q](uit))|(.*(?<=[N|n]o\s)[Q|q]uiero)',Quitar)
Cesta_handler = CommandHandler('Cesta', Cesta)

Yo_handler = CommandHandler('Yo', yo)
Eliminar_handler = CommandHandler('Eliminarme', DarDeBaja)

Cueta_handler = CommandHandler("Cuenta",Cuenta)
SeguirComprando_handler = telegram.ext.RegexHandler("No\. Seguir pidiendo\.",SeguirComprando)
TramitarCompra_handler = telegram.ext.RegexHandler("Sí\. Finalizar pedido\.",TramitarCompra)

In [ ]:
dispatcher = mi_bot_updater.dispatcher

dispatcher.add_handler(start_handler)
dispatcher.add_handler(ayuda_handler)
dispatcher.add_handler(ayuda2_handler)
dispatcher.add_handler(Ejemplos_handler)
dispatcher.add_handler(Pista_handler)

dispatcher.add_handler(RegistrarCliente_handler)
dispatcher.add_handler(MostrarCarta_handler)

dispatcher.add_handler(Foto_handler)
dispatcher.add_handler(Recomendar_handler)

dispatcher.add_handler(Ingredientes_handler)
dispatcher.add_handler(Alergenos_handler)
dispatcher.add_handler(Quitar_handler)
dispatcher.add_handler(Añadir_handler)

dispatcher.add_handler(Cesta_handler)
dispatcher.add_handler(Yo_handler)
dispatcher.add_handler(Eliminar_handler)

dispatcher.add_handler(Cueta_handler)
dispatcher.add_handler(SeguirComprando_handler)
dispatcher.add_handler(TramitarCompra_handler)

In [ ]:
mi_bot_updater.start_polling()

In [ ]:
mi_bot_updater.stop()
dispatcher.handlers = {0: []}
#Clientes = {}

In [ ]:
Clientes

In [ ]:
v = Clientes[393325618]
v.nombre

In [ ]:
np.save('Clientes.npy',Clientes)

In [ ]:
v = np.load('Clientes.npy')

In [ ]:
v